<a href="https://colab.research.google.com/github/chyan0411/Thesis/blob/master/30Matrix_run_file_441_to_9output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install import-ipynb
# import import_ipynb

%tensorflow_version 1.x

# !pip install numpy==1.16.1

TensorFlow 1.x selected.


In [ ]:

import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt 
from keras.models import Model, Sequential
from keras.layers import Conv1D,Flatten,Conv2D, Dropout, MaxPooling2D, Input,Conv3D, MaxPooling3D, Dense, BatchNormalization
from keras import Input,layers
from keras.layers.merge import concatenate 
from google.colab import drive
from keras.optimizers import Adam,RMSprop
from keras.wrappers.scikit_learn import KerasClassifier,KerasRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
drive.mount('/content/drive')
# %cp -av fromfolder tofolder
# %cd "/content/drive/My Drive/Newdata/"
# from model_functions import *

Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:

Data1 = np.load('/content/drive/My Drive/Newdata/matrix_output_con.npy') 
Data2 = np.load('/content/drive/My Drive/Newdata/matrix_output_pre.npy') 
Data3 = np.load('/content/drive/My Drive/Newdata/matrix_output_post.npy') 
Data4 = np.load('/content/drive/My Drive/Newdata/matrix_output_fu.npy') 

# print(Data2.shape)  # 将21*21 的矩阵分成 9个部分 （3*3）
def from_441_to_4(Data):

  Y =[]
  for i in range(24):
    X =[]
    X.append(Data[i,:7,:7])
    X.append(Data[i,:7,7:14])
    X.append(Data[i,:7,14:])
    X.append(Data[i,7:14,:7])
    X.append(Data[i,7:14,7:14])
    X.append(Data[i,7:14,14:])
    X.append(Data[i,14:,:7]) 
    X.append(Data[i,14:,7:14])
    X.append(Data[i,14:,14:])



    Y.append(X)
  Tmp = np.array(Y)
  Tmp = np.reshape(Tmp,(24,9,49))
  print(Tmp.shape)
  return Tmp

To_4_con =  from_441_to_4(Data1)
To_4_pre =  from_441_to_4(Data2)
To_4_post =  from_441_to_4(Data3)
To_4_fu =  from_441_to_4(Data4) 

print(To_4_fu[0,0,0])

(24, 9, 49)
(24, 9, 49)
(24, 9, 49)
(24, 9, 49)
2.0


In [ ]:
# 将分成16份的象限融合，每个象限只用一个数值代表。 0：盲，1：部分盲，2：正常 每个象限用最多值代替（例：假如1最多那就是1）
def change_to_4_dots(Data): # 分成的四个矩阵 每个矩阵用其中数值最多的数来表示（0,1,2）
  Tmp = np.zeros((24,9)) 
  for i in range(24):
    for j in range(9):

      a = np.count_nonzero(Data[i,j] == 0)
      b = np.count_nonzero(Data[i,j] == 1)
      c = np.count_nonzero(Data[i,j] == 2)
      if a == max(a,b,c):
        Tmp[i,j] = 0 
      elif b == max(a,b,c):
        Tmp[i,j] = 1
      elif c == max(a,b,c):
        Tmp[i,j] = 2 
  return Tmp
  
aaa = change_to_4_dots(To_4_con)
bbb = change_to_4_dots(To_4_pre)
ccc = change_to_4_dots(To_4_post)
ddd = change_to_4_dots(To_4_fu)
print(aaa.shape)  #(24, 9)
print(bbb[:3]) 

np.save('/content/drive/My Drive/Newdata/_441to4/To_9_con',aaa)
np.save('/content/drive/My Drive/Newdata/_441to4/To_9_pre',bbb)
np.save('/content/drive/My Drive/Newdata/_441to4/To_9_post',ccc)
np.save('/content/drive/My Drive/Newdata/_441to4/To_9_fu',ddd)

(24, 9)
[[2. 2. 0. 2. 2. 0. 2. 2. 0.]
 [2. 2. 0. 2. 2. 0. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 0.]]


In [ ]:
print(bbb[:3].shape)
a = np.count_nonzero(bbb[:3] == 0)
b = np.count_nonzero(bbb[:3] == 1)
c = np.count_nonzero(bbb[:3] == 2)
print(a)
print(b)
print(c)

(3, 9)
6
0
21


In [ ]:
def from_4_to_3_label(data):
  TMP = []
  for i in range(24):
    Tmp =[]
    for j in range(9):
      tmp = []
      if data[i,j] == 0:
        tmp.append(1) #生成 1,0,0存进list
        tmp.append(0)
        tmp.append(0)
        
      elif data[i,j] == 1:
        tmp.append(0)
        tmp.append(1)
        tmp.append(0)
      
      elif data[i,j] == 2:
        tmp.append(0)
        tmp.append(0)
        tmp.append(1)
      # print(tmp)
      Tmp.append(tmp) 
      # print(len(Tmp))      
    TMP.append(Tmp)
  result = np.array(TMP)
  return result

A = from_4_to_3_label(aaa)
B = from_4_to_3_label(bbb)
C = from_4_to_3_label(ccc)
D = from_4_to_3_label(ddd)
print(B[0,:,:])
print(A.shape)  # (24, 9, 3)

# np.save('/content/drive/My Drive/Newdata/_441to4/To_16_con',A)
# np.save('/content/drive/My Drive/Newdata/_441to4/To_16_pre',B)
# np.save('/content/drive/My Drive/Newdata/_441to4/To_16_post',C)
# np.save('/content/drive/My Drive/Newdata/_441to4/To_16_fu',D)

[[0 0 1]
 [0 0 1]
 [1 0 0]
 [0 0 1]
 [0 0 1]
 [1 0 0]
 [0 0 1]
 [0 0 1]
 [1 0 0]]
(24, 9, 3)


In [ ]:
def make_50_times(data):

  tmp = []
  for i in range(24):
    for j in range(50):
      tmp.append(data[i])
  result = np.array(tmp)
  return result

AA = make_50_times(A)
BB = make_50_times(B)
CC = make_50_times(C)
DD = make_50_times(D)
print(AA.shape)  # (1200, 16, 3)

(1200, 9, 3)


In [ ]:
#该每50个就集成一次 按照病人分类， 24个病人 每个病人150个数据（# pre post fu 每50个 ）
for i in range (24):  
  tmp = np.concatenate((BB[i*50:50*(i+1)],CC[i*50:50*(i+1)],DD[i*50:50*(i+1)]),axis=0)
  np.save('/content/drive/My Drive/Newdata/ordered_by_patient/9/output_'+str(i+1),tmp)
  print(i+1)
  print(tmp.shape)

1
(150, 9, 3)
2
(150, 9, 3)
3
(150, 9, 3)
4
(150, 9, 3)
5
(150, 9, 3)
6
(150, 9, 3)
7
(150, 9, 3)
8
(150, 9, 3)
9
(150, 9, 3)
10
(150, 9, 3)
11
(150, 9, 3)
12
(150, 9, 3)
13
(150, 9, 3)
14
(150, 9, 3)
15
(150, 9, 3)
16
(150, 9, 3)
17
(150, 9, 3)
18
(150, 9, 3)
19
(150, 9, 3)
20
(150, 9, 3)
21
(150, 9, 3)
22
(150, 9, 3)
23
(150, 9, 3)
24
(150, 9, 3)


In [ ]:
patient_order = range(1,25)
order = shuffle(patient_order, random_state = 10)
print(patient_order)
print(order)  
# [14, 18, 20, 5, 2, 22, 19, 15, 23, 17, 24, 13, 11, 6, 9, 7, 1, 3, 8, 12, 21, 10, 16, 4]    random_state =20 
# [6, 2, 8, 3, 12, 13, 22, 14, 15, 4, 23, 7, 19, 11, 21, 24, 9, 20, 17, 18, 1, 16, 5, 10]    random_state =10

range(1, 25)
[6, 2, 8, 3, 12, 13, 22, 14, 15, 4, 23, 7, 19, 11, 21, 24, 9, 20, 17, 18, 1, 16, 5, 10]


In [ ]:
def reshape(Num):   #把 30,150,116,116变成  150,30,116,116

  data = np.load('/content/drive/My Drive/Newdata/ordered_by_patient/' + str(Num)+ '.npy') 
  data = np.reshape(data,(150,30,116,116))
  np.save('/content/drive/My Drive/Newdata/ordered_by_patient/115_30_116_116_' + str(Num),data)
  return data 

In [ ]:
tmp = order[0]
A = np.load('/content/drive/My Drive/Newdata/ordered_by_patient/115_30_116_116_' + str(tmp)+ '.npy')
for i in order[1:18]: 
  a = np.load('/content/drive/My Drive/Newdata/ordered_by_patient/115_30_116_116_' + str(i)+ '.npy')
  A = np.concatenate((A,a), axis = 0)
print(A.shape)
# 前十八个 病人数据做训练data(2700, 30, 116, 116)

(2700, 30, 116, 116)


In [ ]:

tmp = order[0]
B = np.load('/content/drive/My Drive/Newdata/ordered_by_patient/9/output_' + str(tmp)+ '.npy')
for i in order[1:18]: 
  b = np.load('/content/drive/My Drive/Newdata/ordered_by_patient/9/output_' + str(i)+ '.npy')
  B = np.concatenate((B,b), axis = 0)
print(B.shape)
# 前十八个 病人数据做训练label(2700, 9, 3)

(2700, 9, 3)


In [ ]:
tmp = order[18]
C = np.load('/content/drive/My Drive/Newdata/ordered_by_patient/115_30_116_116_' + str(tmp)+ '.npy')
for i in order[19:]: 
  c = np.load('/content/drive/My Drive/Newdata/ordered_by_patient/115_30_116_116_' + str(i)+ '.npy')
  C = np.concatenate((C,c), axis = 0)
print(C.shape)
# 后六个人 病人数据做测试data(900, 30, 116, 116)

(900, 30, 116, 116)


In [ ]:
tmp = order[18]
D = np.load('/content/drive/My Drive/Newdata/ordered_by_patient/9/output_' + str(tmp)+ '.npy')
for i in order[19:]: 
  d = np.load('/content/drive/My Drive/Newdata/ordered_by_patient/9/output_' + str(i)+ '.npy')
  D = np.concatenate((D,d), axis = 0)
print(D.shape)
# 后六个人 病人数据做测试label(900, 9, 3)

(900, 9, 3)


In [ ]:
# np.save('/content/drive/My Drive/Newdata/ordered_by_patient/train_data_2700_30_116_116', A)
np.save('/content/drive/My Drive/Newdata/ordered_by_patient/train_label_2700_9_3', B)
# np.save('/content/drive/My Drive/Newdata/ordered_by_patient/test_data_900_30_116_116', C)
np.save('/content/drive/My Drive/Newdata/ordered_by_patient/test_label_900_9_3', D)